Libs

In [ ]:
!pip install pyvirtualdisplay
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, classification_report
from keras.models import Sequential
from keras.layers import Dense, Dropout
import gym
import io
import base64
from pyvirtualdisplay import Display
from gym.wrappers.record_video import RecordVideo
import glob
from IPython.display import display, HTML

Train test e BD

In [ ]:
# Carregar o dataset
novo_dados_path = '/content/sample_data/BD_MontainCar.csv'
novo_dados = pd.read_csv(novo_dados_path)

# Remover colunas desnecessárias
novo_dados = novo_dados.drop(columns=["Unnamed: 0"])

# Separar previsores e classes
novo_previsores = novo_dados[['Posicao', 'Velocidade']]
novo_classe = novo_dados['Acao']

# Normalizar os previsores
novo_scaler = MinMaxScaler()
novo_previsores = novo_scaler.fit_transform(novo_previsores)

# Dividir dados em treino e teste
novo_previsores_treinamento, novo_previsores_teste, novo_classe_treinamento, novo_classe_teste = train_test_split(
    novo_previsores, novo_classe, test_size=0.30, random_state=42
)

Modelo

In [ ]:
# Criar o modelo
novo_classificador = Sequential()
novo_classificador.add(Dense(units=64, input_dim=2, kernel_initializer='random_uniform', activation='relu'))
novo_classificador.add(Dropout(0.2))
novo_classificador.add(Dense(units=32, kernel_initializer='random_uniform', activation='relu'))
novo_classificador.add(Dropout(0.2))
novo_classificador.add(Dense(units=16, kernel_initializer='random_uniform', activation='relu'))
novo_classificador.add(Dense(units=3, kernel_initializer='random_uniform', activation='softmax'))
novo_classificador.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Treinar o modelo
novo_classificador.fit(novo_previsores_treinamento, novo_classe_treinamento, batch_size=20, epochs=150)

Metricas

In [ ]:
# Avaliar a acurácia
_, novo_accuracy = novo_classificador.evaluate(novo_previsores_teste, novo_classe_teste)
print(f"Acurácia no conjunto de teste: {novo_accuracy:.2f}")

# Previsões no conjunto de teste
novo_previsoes = novo_classificador.predict(novo_previsores_teste)
novo_previsoes_classes = np.argmax(novo_previsoes, axis=1)

# Matriz de confusão
novo_matriz_confusao = confusion_matrix(novo_classe_teste, novo_previsoes_classes)
plt.figure(figsize=(8, 6))
sns.heatmap(novo_matriz_confusao, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Classe Prevista')
plt.ylabel('Classe Real')
plt.title('Matriz de Confusão')
plt.show()

# Relatório de classificação
print("Relatório de Classificação:")
print(classification_report(novo_classe_teste, novo_previsoes_classes))


Resultado

In [ ]:
warnings.filterwarnings("ignore", category=UserWarning)
# Funções para avaliar o desempenho no ambiente MountainCar
def avaliar_ambiente(modelo, env, scaler, episodios=1):
    total_recompensas = []
    for _ in range(episodios):
        obs = env.reset()
        total_reward = 0
        done = False
        while not done:
            entrada = scaler.transform(np.array([obs]))
            acao_probabilidades = modelo.predict(entrada, verbose=1)
            acao = np.argmax(acao_probabilidades)
            obs, reward, done, _ = env.step(acao)
            total_reward += reward
        total_recompensas.append(total_reward)
    return np.mean(total_recompensas)

def avaliar_passos(modelo, env, scaler, episodios=10):
    total_passos = []
    for _ in range(episodios):
        obs = env.reset()
        passos = 0
        done = False
        while not done:
            entrada = scaler.transform(np.array([obs]))
            acao_probabilidades = modelo.predict(entrada, verbose=0)
            acao = np.argmax(acao_probabilidades)
            obs, _, done, _ = env.step(acao)
            passos += 1
        total_passos.append(passos)
    return np.mean(total_passos)

# Função para gravar e exibir o vídeo da execução
def show_video():
    mp4list = glob.glob('video/*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[-1]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data=f'''<video alt="test" autoplay
                              controls style="height: 400px;">
                              <source src="data:video/mp4;base64,{encoded.decode('ascii')}" type="video/mp4" />
                           </video>'''))
    else:
        print("Não foi possível encontrar o vídeo.")

# Avaliar o desempenho no ambiente
env = RecordVideo(gym.make('MountainCar-v0'), './video')
media_recompensa = avaliar_ambiente(novo_classificador, env, novo_scaler)
media_passos = avaliar_passos(novo_classificador, env, novo_scaler)

print(f"Média de recompensa em 10 episódios: {media_recompensa:.2f}")
print(f"Média de passos para completar o objetivo: {media_passos:.2f}")

# Finalizar e exibir o vídeo
env.close()
show_video()

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/monitoring/video_recorder.py:78: DeprecationWarning: WARN: Recording ability for environment MountainCar-v0 initialized with `render_mode=None` is marked as deprecated and will be removed in the future.
  logger.deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step


/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: htt

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step


/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━